In [35]:
!rm ~/Downloads/*.csv

zsh:1: no matches found: /Users/leswing/Downloads/*.csv


In [20]:
# Save https://www.fantasypros.com/nfl/projections/qb.php?week=draft in folder "fantasy"
# Save https://www.fantasypros.com/nfl/adp/overall.php in folder "fantasy"
!mv /Users/leswing/Downloads/*.csv ./fantasy

In [2]:
import json
import pandas as pd

In [3]:
data = open('../app/public/projections.json').read()
d = json.loads(data)

In [24]:
bye_weeks = {
    "DET": 8,
    "LAC": 12,
    "PHI": 9,
    "TEN": 10,
    "KC": 10,
    "LAR": 8,
    "MIA": 12,
    "MIN": 6,
    "CHI": 5,
    "DAL": 10,
    "PIT": 5,
    "SF": 14,
    "CLE": 9,
    "GB": 5,
    "LV": 8,
    "SEA": 8,
    "ARI": 8,
    "CAR": 14,
    "NYG": 14,
    "TB": 9,
    "ATL": 5,
    "BUF": 7,
    "CIN": 10,
    "JAC": 8,
    "NO": 11,
    "NYJ": 9,
    "BAL": 7,
    "DEN": 12,
    "HOU": 6,
    "NE": 14,
    "WAS": 12,
    "IND": 11
}

def blank_player():
    # TODO fill out key, index, std, halfPpr, ppr
    return {
        'index': None,
        'key': None,
        'name': None,
        'pos': None,
        'team': None,
        'bye': 1,
        'std': None,
        'halfPpr': None,
        'ppr': None,
        'passYds': None,
        'rushTds': None,
        'dfSafeties': None,
        'kick4049': None,
        'dfSacks': None,
        'kick2029': None,
        'passInts': None,
        'rushYds': None,
        'receptionTds': None,
        'receptions': None,
        'dfInts': None,
        'kickExtraPoints': None,
        'passTds': None,
        'kick50': None,
        'fumbles': None,
        'kick019': None,
        'receptionYds': None,
        'twoPts': None,
        'dfPointsAllowedPerGame': None,
        'kick3039': None,
        'dfFumbles': None,
        'dfTds': None
    }
adp = pd.read_csv("fantasy/FantasyPros_2025_Draft_ALL_Rankings.csv")
adp = adp.dropna()
adp_lookup = {}
for _, row in adp.iterrows():
    key = f"{row['PLAYER NAME']}-{row['TEAM']}"
    if key not in adp_lookup:
        adp_lookup[key] = row['RK']

In [38]:
def map_from_csv(df, column_mapping):
    qb_players = []
    for _, row in df.iterrows():
        new_player = blank_player()
        for k, v in row.items():
            if k not in column_mapping:
                continue
            new_key = column_mapping[k]
            try:
                my_v = str(v).replace(',','')
                new_player[new_key] = float(my_v)
            except:
                new_player[new_key] = str(v)
        qb_players.append(new_player)
    return qb_players

def get_qbs():
    map_qb = {
        "PLAYER NAME": "name",
        "TEAM": "team",
        "YDS": "passYds",
        "TDS": "passTds",
        "INT": "passInts",
        "YDS.1": "rushYds",
        "TDS.1": "rushTds",
    }
    df = pd.read_csv("fantasy/FantasyPros_2025_Draft_QB_Rankings.csv")
    df = df.dropna()
    df['POS'] = 'QB'
    return map_from_csv(df, map_qb)

def get_rbs():
    rb_map = {
        "PLAYER NAME": "name",
        "TEAM": "team",
        "YDS": "rushYds",
        "TDS": 'rushTds',
        "REC": "receptions",
        "YDS.1": "receptionYds",
        "TDS.1": "receptionTds",
    }
    df = pd.read_csv("fantasy/FantasyPros_2025_Draft_RB_Rankings.csv")
    df = df.dropna()
    df['POS'] = "RB"
    return map_from_csv(df, rb_map)

def get_wrs():
    wr_map = {
        "PLAYER NAME": "name",
        "TEAM": "team",
        "YDS.1": "rushYds",
        "TDS.1": 'rushTds',
        "REC": "receptions",
        "YDS": "receptionYds",
        "TDS": "receptionTds",
    }
    df = pd.read_csv("fantasy/FantasyPros_2025_Draft_WR_Rankings.csv")
    df = df.dropna()
    df['POS'] = 'WR'
    return map_from_csv(df, wr_map)

def get_tes():
    te_map = {
        "PLAYER NAME": "name",
        "TEAM": "team",
        "REC": "receptions",
        "YDS": "receptionYds",
        "FL": "fumbles"
    }
    df = pd.read_csv("fantasy/FantasyPros_2025_Draft_TE_Rankings.csv")
    df = df.dropna()
    df['POS'] = 'TE'
    return map_from_csv(df, te_map)

def get_ks():
    k_map = {
        "PLAYER NAME": "name",
        "TEAM": "team",
        "XPM": "kickExtraPoints",
        "FG": "30-39"
    }
    df = pd.read_csv("fantasy/FantasyPros_2025_Draft_K_Rankings.csv")
    df = df.dropna()
    df['POS'] = 'K'
    return map_from_csv(df, k_map)

def get_ds():
    dst_map = {
        "PLAYER NAME": "name",
        "TEAM": "team",
        "SACK": "dfSacks",
        "INT": "dfInts",
        "FUMR": "dfFumbles",
        "DEF TD": "dfTds",
    }
    df = pd.read_csv("fantasy/FantasyPros_2025_Draft_DST_Rankings.csv")
    df = df.drop("TEAM", axis=1)
    df = df.dropna()
    df['POS'] = 'DST'
    df = map_from_csv(df, dst_map)
    return df

def get_adp_for_player(p):
    key = f"{p['name']}-{p['team']}"
    if key not in adp_lookup:
        return 1_000
    return adp_lookup[key]

In [39]:
qb_players = get_qbs()
rb_players = get_rbs()
wr_players = get_wrs()
te_players = get_tes()
k_players = get_ks()
dst_players = get_ds()
[len(x) for x in [qb_players, rb_players, wr_players, te_players, k_players, dst_players]]

[78, 145, 188, 101, 44, 32]

In [40]:
all_players = []
all_players.extend(qb_players)
all_players.extend(rb_players)
all_players.extend(wr_players)
all_players.extend(te_players)
all_players.extend(k_players)
all_players.extend(dst_players)

In [41]:
import uuid
#key, index, std, halfPpr, ppr
for i, p in enumerate(all_players):
    p['key'] = str(uuid.uuid4())
    p['index'] = i + 1
    p['std'] = get_adp_for_player(p)
    p['halfPpr'] = get_adp_for_player(p)
    p['ppr'] = get_adp_for_player(p)
    try:
        p['bye'] = bye_weeks[p['team']]
    except:
        pass

In [42]:
all_players = sorted(all_players, key=lambda x: x['ppr'])
d['data'] = all_players
len(all_players)

588

In [43]:
for p in all_players:
    if pd.isna(p['team']):
        p['team'] = 'unrostered'

In [44]:
with open("projections.json", 'w') as fout:
    fout.write(json.dumps(d, indent=4))

In [83]:
df = pd.read_csv("fantasy/FantasyPros_2025_DST_ADP_Rankings.csv")
df = df.drop("Team", axis=1)

In [17]:
rb_players

[{'index': 29,
  'key': '5e881d2a-7dd7-492a-9ae0-576baa7c445b',
  'name': 'Saquon Barkley',
  'pos': 'RB',
  'team': 'PHI',
  'bye': 9,
  'std': 1000,
  'halfPpr': 1000,
  'ppr': 1000,
  'passYds': None,
  'rushTds': None,
  'dfSafeties': None,
  'kick4049': None,
  'dfSacks': None,
  'kick2029': None,
  'passInts': None,
  'rushYds': None,
  'receptionTds': None,
  'receptions': None,
  'dfInts': None,
  'kickExtraPoints': None,
  'passTds': None,
  'kick50': None,
  'fumbles': None,
  'kick019': None,
  'receptionYds': None,
  'twoPts': None,
  'dfPointsAllowedPerGame': None,
  'kick3039': None,
  'dfFumbles': None,
  'dfTds': None},
 {'index': 30,
  'key': 'bacd87ee-171e-4b66-9286-c88feba7450c',
  'name': 'Bijan Robinson',
  'pos': 'RB',
  'team': 'ATL',
  'bye': 5,
  'std': 1000,
  'halfPpr': 1000,
  'ppr': 1000,
  'passYds': None,
  'rushTds': None,
  'dfSafeties': None,
  'kick4049': None,
  'dfSacks': None,
  'kick2029': None,
  'passInts': None,
  'rushYds': None,
  'receptionT

In [93]:
dst_players

[{'index': 304,
  'key': '25fb59bc-72f6-4d50-a803-078f0643be2f',
  'name': 'Denver Broncos ',
  'pos': 'DST',
  'team': None,
  'bye': 1,
  'std': 1000,
  'halfPpr': 1000,
  'ppr': 1000,
  'passYds': None,
  'rushTds': None,
  'dfSafeties': None,
  'kick4049': None,
  'dfSacks': None,
  'kick2029': None,
  'passInts': None,
  'rushYds': None,
  'receptionTds': None,
  'receptions': None,
  'dfInts': None,
  'kickExtraPoints': None,
  'passTds': None,
  'kick50': None,
  'fumbles': None,
  'kick019': None,
  'receptionYds': None,
  'twoPts': None,
  'dfPointsAllowedPerGame': None,
  'kick3039': None,
  'dfFumbles': None,
  'dfTds': None},
 {'index': 305,
  'key': '4a7d618d-f6e8-4039-8f3a-e967299326ee',
  'name': 'Philadelphia Eagles ',
  'pos': 'DST',
  'team': None,
  'bye': 1,
  'std': 1000,
  'halfPpr': 1000,
  'ppr': 1000,
  'passYds': None,
  'rushTds': None,
  'dfSafeties': None,
  'kick4049': None,
  'dfSacks': None,
  'kick2029': None,
  'passInts': None,
  'rushYds': None,
  'r